In [ ]:
import numpy as np 
import pandas as pd 
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.layers import Dense, Lambda,Embedding, Conv1D,Conv2D
from tensorflow.keras.layers import MaxPool1D,Dropout, Lambda, Concatenate, Flatten
from tensorflow.keras.layers import Multiply,Add
import matplotlib.pyplot as plt


In [ ]:
class dataprep:
  def __init__(self,num_targets, num_negs,seq_len):
    self.num_targets = num_targets
    self.num_negs = num_negs
    self.seq_len = seq_len

  def sequence_df(self):
    url = 'https://raw.githubusercontent.com/malinphy/datasets/main/ml100K/ratings.csv'
    df = pd.read_csv(url)
    df.rename(columns = {'userId':'user_id', 'movieId':'item_id'}, inplace = True)
  # df
    df['timestamp']= pd.to_datetime(df['timestamp'], unit = 's')
    df.head(10)

    def id_converter(df,x):
      unique_params = df[x].unique()
      num_unique = len(unique_params)
      param_2enc = { i:j for i,j in enumerate(unique_params)}
      enc_2param = { j:i for i,j in enumerate(unique_params)}


      return num_unique , param_2enc, enc_2param

    num_users, user_2enc ,enc_2user = id_converter(df,'user_id')
    num_items, item_2enc ,enc_2item = id_converter(df,'item_id')

    df['user_id'] = df['user_id'].map(enc_2user)
    df['item_id'] = df['item_id'].map(enc_2item)

    def time_sorter(df):
    ### users and releated items were sorted according to time as function of increasing time 
      return(df.set_index(['user_id','timestamp']).sort_index().reset_index())

    time_sorted_df = time_sorter(df)
    time_sorted_df = time_sorted_df.drop(columns= ['timestamp','rating'])   
    time_sorted_df.head(2)

    def users_grouped(df,x):
      return df.groupby([x]).aggregate(lambda tdf: tdf.unique().tolist()).reset_index()
    # df = df.reset_index()
    
    grouped_df = users_grouped(time_sorted_df,'user_id')
    grouped_df.head(3)

    # def sequence_creator(df, self.seq_len, self.num_targets):
    def sequence_creator(df):
      users = []
      seqs = []
      targets = []

      for i in range(len(grouped_df)):

        for j in range(0,len(grouped_df['item_id'][i])-self.seq_len,self.num_targets):
          users.append(i)
          x = grouped_df['item_id'][i][j:j+self.seq_len+self.num_targets]
          x1 = grouped_df['item_id'][i][j:j+self.seq_len]
          seqs.append(list(x[0:self.seq_len]))
          targets.append(np.array(x[-self.num_targets:]))

      return (pd.DataFrame({'users':users,'sequences':seqs, 'targets':targets}))

    L = 5
    d = 2
    sequenced_df = sequence_creator(grouped_df)
    sequenced_df.head(2)

    d = 2

    tot2 = []
    for j in range(self.num_negs):
      tot = []
      for i in range(len(sequenced_df)):
        sampled_neg = np.random.randint(num_items)
        x  = sequenced_df['users'][i]
        grouped_df['item_id'][x]
        while ((sampled_neg in sequenced_df['sequences'][i]) and  (sampled_neg in sequenced_df['sequences'][i] in grouped_df['item_id'][x] ) ):
          sampled_neg = np.random.randint(num_items)
        tot.append(sampled_neg)
  
      tot2.append(np.array(tot))

    negs = np.array(tot2).T

    sequenced_df['negs'] = list(negs)


    return sequenced_df, num_users, num_items

x = dataprep(1,1,5)
df,num_users, num_items = x.sequence_df()
df.head(1)

,users,sequences,targets,negs
0,0,"[16, 17, 19, 8, 9]",[10],[4414]


In [ ]:
def expander1(x):
    return (x[:,:,:,tf.newaxis])
    
def seqer2(x):
    return (tf.squeeze(x, axis=2))

def seqer1(x):
    return (tf.squeeze(x, axis=1))

TARGET_NUM = 1
batch_size = 4096
L=5
d=16
d_prime=4 
drop_ratio=0.05
num_factors = 5
# num_factors = 10
dims = num_factors

In [ ]:
doublet = np.squeeze(np.dstack([df['targets'].astype('int'), df['negs'].astype('int')]))
k = []
sayac = 1
for i in df['sequences']:
  t =[]
  for j in i:
    t.append(int(j))
  k.append(t)

y_dummy = tf.ones( [len(df['users']),1])

In [ ]:
user_input = tf.keras.Input(shape = (1,), name = 'user_input')  ### kullanildi
print('user_input_shape:',user_input.shape)
seq_input = tf.keras.Input(shape = (L,), name = 'seq_input')  ### kullanildi
print('seq_input_shape:',seq_input.shape)
# doublet_input = tf.keras.Input(shape = (None,), name = 'doublet_input')
doublet_input = tf.keras.Input(shape = (2,), name = 'doublet_input')
print('dobulet_input_shape:',doublet_input.shape)
seq_embedding = Embedding(num_items,num_factors, name = 'sequence_embedding')(seq_input)
print('seq_embedding_shape:',seq_embedding.shape)
seq_embedding = tf.expand_dims(seq_embedding, 3, name = 'sequence_embedding_expander')
print('seq_embedding_expander_shape:',seq_embedding.shape)
doublet_embedding = Embedding(num_items, num_factors*2,
                              name = 'doublet_embedding')(doublet_input)## buyuk W
print('doublet_embedding_shape:',doublet_embedding.shape)
user_embedding = Embedding(num_users, num_factors,
                               name = 'user_embedding')(user_input)
print('user_embedding_shape:',user_embedding.shape)
h = [i + 1 for i in range(L)]
fc1_dim_v = d_prime * num_factors
print('fc1_dim_v:',fc1_dim_v)
out_v = Conv2D(d_prime,(L,1), name = 'convo_v')(seq_embedding)
print('out_v_1_shape:',out_v.shape)
out_v = Flatten(name = 'flatten_convo_v')(out_v)
print('out_v_2_shape:',out_v.shape)
out_hs = []
for i in (h):
  seq_mod = tf.keras.Sequential([
           Conv2D(d, (i, num_factors)),
           Lambda(seqer2),
           MaxPool1D(L - i + 1),
           Lambda(seqer1)                      
          ])
  out_hs.append(seq_mod(seq_embedding))
# print('out_hs_shape:',np.array(out_hs).shape)
out_h = tf.concat(out_hs, axis=1, name ='sequential_out') # galiba axis 2 olmali
print('out_h_2_shape:',out_h.shape)
out = tf.concat([out_v, out_h], axis = 1, name ='convo_concat')
print('out_shape:',out.shape)
dropout_layer = Dropout(0.05, name = 'dropout_layer')(out)
z = Dense(dims, name = 'dense_layer', activation = 'relu')(dropout_layer)
print('z_shape:',z.shape)
flat_users = Flatten(name = 'flatten')(user_embedding)
print('flat_users_shape:',flat_users.shape)
x = tf.concat([z, flat_users], axis=1, name = 'x')
print('x_shape:',x.shape)
res = tf.matmul(doublet_embedding,tf.expand_dims(x,axis=-1))
print('res_shape:',res.shape)
doublet_embedding_b = Embedding(num_items, 1,
                               name = 'doublet_embedding_b')(doublet_input)## kucuk b  
res = res+doublet_embedding_b
# res = tf.math.reduce_sum(res, axis = 1)
res = tf.squeeze(res,axis =2)
@tf.function
def identity_loss(y_target, y_pred):
# def identity_loss(y_target, y_pred):  
  pos, neg = tf.split(y_pred,2,1)
  positive_loss = -1*tf.math.reduce_mean(tf.math.log(tf.sigmoid(pos)))
  negative_loss = -1*tf.math.reduce_mean(tf.math.log(1- tf.sigmoid(neg)))
  total_loss = tf.math.add(positive_loss, negative_loss)
  # return tf.math.reduce_mean(y_pred)
  return total_loss
caser_model =tf.keras.Model(inputs= [
                                      user_input,
                                      seq_input,
                                      doublet_input
                                      ],
                            outputs = res,
                            # outputs = x
                            )

caser_model.compile(loss = identity_loss,optimizer = 'Adam')

caser_hist = caser_model.fit(
                            [
                              tf.constant(df['users']),
                              tf.constant(k),
                              tf.constant( np.squeeze( np.dstack( [ df[ 'targets'].astype('int'),
                                                                   df['negs'].astype('int') 
                                                                   ] ) ) ) ],
                y_dummy,
                epochs = 2,  
                batch_size = 512 
                )


user_input_shape: (None, 1)
seq_input_shape: (None, 5)
dobulet_input_shape: (None, 2)
seq_embedding_shape: (None, 5, 5)
seq_embedding_expander_shape: (None, 5, 5, 1)
doublet_embedding_shape: (None, 2, 10)
user_embedding_shape: (None, 1, 5)
fc1_dim_v: 20
out_v_1_shape: (None, 1, 5, 4)
out_v_2_shape: (None, 20)
out_h_2_shape: (None, 80)
out_shape: (None, 100)
z_shape: (None, 5)
flat_users_shape: (None, 5)
x_shape: (None, 10)
res_shape: (None, 2, 1)
Epoch 1/2
189/189 [==============================] - 11s 30ms/step - loss: 1.2008
Epoch 2/2
189/189 [==============================] - 4s 23ms/step - loss: 0.9319


In [ ]:
# ### pred for pytorch
# doublet_embedding_pos , doublet_embedding_neg = tf.split(doublet_embedding,2, axis = 1)
# doublet_embedding_b_pos , doublet_embedding_b_neg = tf.split(doublet_embedding_b,2, axis = 1)
# tf.math.reduce_sum((tf.squeeze(x, axis = 0) * doublet_embedding_pos), axis = 1
#                    ) + doublet_embedding_b_pos 


# tf.math.reduce_sum((x * tf.squeeze(doublet_embedding_pos, axis=1)), axis = 1
#                    ) + tf.squeeze(doublet_embedding_b_pos, axis =1 )
# print('out_v', out_v.shape)
# print('out_h', out_h.shape)
# print('out', out.shape)
# print('X',x.shape)
# print('user_embedding',user_embedding.shape)
# print('seq_embedding',seq_embedding.shape)
# print('doublet_embedding',doublet_embedding.shape)
# print('doublet_embedding_b',doublet_embedding_b.shape)
# print('z',z.shape)
# print('x',x.shape)
# print('doublet_embedding_pos',doublet_embedding_pos.shape)
# print('doublet_embedding_pos',tf.squeeze(doublet_embedding_b_pos, axis = 1).shape)

In [ ]:
#### Since model has custom loss which is called as troplet loss
#### it is not possible to load the entire model without compiling after 
#### loading the model.
#### To overcome this issue, we will keep the model and save the weights.


In [ ]:
# for layer in caser_model.layers:
#   print(layer.name, layer)

In [ ]:
doublet_embedding_weights = caser_model.get_layer('doublet_embedding').weights  ## shape=(9066, 20) split atabilirim
doublet_embedding_b_weights = caser_model.get_layer('doublet_embedding_b').weights  ## shape=(9066, 1)
dense_layer_weights = caser_model.get_layer('dense_layer').weights ## k1 : shape=(120, 10) , k2 : shape=(10,)
flat_layer_weights = caser_model.get_layer('flatten').weights ## []
user_embedding_weights = caser_model.get_layer('user_embedding').weights  ## shape=(671, 10)
flat_users=tf.reshape(user_embedding_weights, 
           [tf.shape(user_embedding_weights)[1]*tf.shape(user_embedding_weights)[2]])

In [ ]:
dense_layer_weights = caser_model.get_layer('dense_layer').weights ## k1 : shape=(120, 10) , k2 : shape=(10,)  Z
user_embedding_weights = caser_model.get_layer('user_embedding').weights  ## shape=(671, 10)

user_embedding_weights_f = tf.reshape(user_embedding_weights, tf.shape(user_embedding_weights)[0]*
                                                              tf.shape(user_embedding_weights)[1]*
                                                              tf.shape(user_embedding_weights)[2])

In [ ]:
preds = tf.math.reduce_sum(tf.concat([user_embedding_weights_f, dense_layer_weights[1]],axis = 0), axis =0)*doublet_embedding_b_weights

In [ ]:
# tf.squeeze(preds)

In [ ]:
print(np.array(dense_layer_weights).shape)
print(np.array(user_embedding_weights).shape)
print(np.array(user_embedding_weights_f).shape)

(2,)
(1, 671, 5)
(3355,)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [ ]:
caser_model.save_weights('drive/MyDrive/caser_model_weights')
caser_model.load_weights('drive/MyDrive/caser_model_weights')
# caser_model.summary()

In [ ]:
def predict(model):
  dense_layer_weights = model.get_layer('dense_layer').weights ## k1 : shape=(120, 10) , k2 : shape=(10,)  Z
  user_embedding_weights = model.get_layer('user_embedding').weights  ## shape=(671, 10)
  doublet_embedding_b_weights = model.get_layer('doublet_embedding_1').weights
  user_embedding_weights_f = tf.reshape(user_embedding_weights, tf.shape(user_embedding_weights)[0]*
                                                              tf.shape(user_embedding_weights)[1]*
                                                              tf.shape(user_embedding_weights)[2])
  preds = tf.math.reduce_sum(tf.concat([user_embedding_weights_f, dense_layer_weights[1]],axis = 0), axis =0)*doublet_embedding_b_weights
  x = tf.squeeze(preds)

  return x

In [ ]:
caser_model.layers[-2].output

<KerasTensor: shape=(None, 2, 1) dtype=float32 (created by layer 'tf.__operators__.add')>

In [ ]:
caser_model.get_layer('dense_layer')

In [ ]:
dense_layer_weights = caser_model.get_layer('dense_layer').weights ## k1 : shape=(120, 10) , k2 : shape=(10,)  Z
user_embedding_weights = caser_model.get_layer('user_embedding').weights  ## shape=(671, 10)
doublet_embedding_b_weights = caser_model.get_layer('doublet_embedding_b').weights
user_embedding_weights_f = tf.reshape(user_embedding_weights, tf.shape(user_embedding_weights)[0]*
                                                              tf.shape(user_embedding_weights)[1]*
                                                              tf.shape(user_embedding_weights)[2])
preds = tf.math.reduce_sum(tf.concat([user_embedding_weights_f, dense_layer_weights[1]],axis = 0), axis =0)*doublet_embedding_b_weights
x = tf.squeeze(preds)

In [ ]:
doublet_embedding_b_model = Model(inputs = caser_model.input,
                         outputs = caser_model.layers[-3].output)
# doublet_embedding_b_model.summary()
doublet_embedding_w_model = Model(inputs = caser_model.input,
                         outputs = caser_model.layers[-6].output)

# doublet_embedding_w_model.summary()
x_model = Model(inputs = caser_model.input,
                         outputs = caser_model.layers[-7].output)

# x_model.summary()

In [ ]:
doublet_embedding_w_model_out = doublet_embedding_w_model([
                              tf.constant(df['users']),
                              tf.constant(k),
                              tf.constant( np.squeeze( np.dstack( [ df[ 'targets'].astype('int'),
                                                                   df['negs'].astype('int') 
                                                                   ] ) ) ) ])

doublet_embedding_b_model_out = doublet_embedding_b_model([
                              tf.constant(df['users']),
                              tf.constant(k),
                              tf.constant( np.squeeze( np.dstack( [ df[ 'targets'].astype('int'),
                                                                   df['negs'].astype('int') 
                                                                   ] ) ) ) ])

x_model_out = x_model([
                              tf.constant(df['users']),
                              tf.constant(k),
                              tf.constant( np.squeeze( np.dstack( [ df[ 'targets'].astype('int'),
                                                                   df['negs'].astype('int') 
                                                                   ] ) ) ) ])

In [ ]:
print(doublet_embedding_w_model_out.shape) #### belki buna ortadan split atabilirim
print(doublet_embedding_b_model_out.shape) #### belki buna ortadan split atabilirim
print(x_model_out.shape) #### belki buna ortadan split atabilirim

(96649, 2, 10)
(96649, 2, 1)
(96649, 10)


In [ ]:
print(np.array(dense_layer_weights).shape)
print(np.array(user_embedding_weights).shape)
print(np.array(user_embedding_weights_f).shape)

(2,)
(1, 671, 5)
(3355,)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [ ]:
print(np.array(tf.split(doublet_embedding_w_model_out,2,axis = 1)[0]).shape) #### belki buna ortadan split atabilirim
print(np.array(tf.split(doublet_embedding_b_model_out,2,axis =1)[0]).shape) #### belki buna ortadan split atabilirim
print(x_model_out.shape) #### belki buna ortadan split atabilirim

(96649, 1, 10)
(96649, 1, 1)
(96649, 10)


In [ ]:
#### prediction function
w2_pred = tf.squeeze(tf.split(doublet_embedding_w_model_out,2,
                              axis = 1)[0], axis = 1)
b2_pred = tf.squeeze(tf.squeeze(tf.split(doublet_embedding_b_model_out,2,
                              axis =1)[0], axis = 1), axis =1)
# tf.matmul(x,)
print(w2_pred.shape)
print(b2_pred.shape)

(96649, 10)
(96649,)


In [ ]:
w2_pred.shape

TensorShape([96649, 10])

In [ ]:
finalo = tf.math.reduce_sum((x_model_out*w2_pred) ,axis =1) + b2_pred

<tf.Tensor: shape=(96649,), dtype=float32, numpy=
array([1.8717275, 1.1254894, 1.6261234, ..., 0.4317628, 1.0245713,
       1.1817822], dtype=float32)>